# Challenge Description
Company XYZ is an Online Travel Agent site, such as Expedia, Booking.com, etc. They haven't invested in data science yet and all the data they have about user searches are simply stored in the URLs generated when users search for a hotel. If you are not familiar with URLs, you can run a search on any OTA site and see how all search parameters are present in the URL. 

# Question 1
Create a clean data set where each column is a ﬁeld in the URL, each row is a given search and the cells are the corresponding URL values. 

In [1]:
import pandas as pd
import numpy as np

In [17]:
file="data/url_list.txt"
#url=pd.read_csv(file)

below answer is from https://github.com/stasi009/TakeHomeDataChallenges/blob/master/16.ParseURL/parse_url.ipynb

In [5]:
# constant definition
Site = 'http://www.mysearchforhotels.com/shop/hotelsearch?'
LenSite = len(Site)

ParamPrefix = 'hotel.'
LenParaPrefix = len(ParamPrefix)

Separator = ', '

In [7]:
def parse_url(url):
    """
    input: a url string
    output: a dictionary which contains parameter name and its value
    """
    # remove common prefix
    assert url[LenSite-1] == '?'
    segments = url[LenSite:].split('&')

    params = {}
    for segment in segments:
        kvpairs = segment.split('=')
        assert len(kvpairs) == 2

        k = kvpairs[0]
        # remove common prefix
        assert k[LenParaPrefix-1] == '.'
        k = k[LenParaPrefix:]

        if k in params:
            print("'{}' has already existed in search".format(k))
            params[k] = params[k] + Separator +kvpairs[1]
        else:
            params[k] = kvpairs[1]

    return params

In [12]:
def load_parse():
    succ_urls = []
    fail_urls = []
    with open("data/url_list.txt",'rt') as inf:
        for index,line in enumerate(inf):
            try:
                url = parse_url(line.strip())
                succ_urls.append(url)
            except:
                fail_urls.append(line)
                print("failed to parse: {}".format(line))

            # if index%1000 ==0: print '{} lines parsed'.format(index)

    print("************ ALL DONE ************")
    return succ_urls,fail_urls

In [14]:
succ_urls,fail_urls = load_parse()


'amenities' has already existed in search
'amenities' has already existed in search
'amenities' has already existed in search
'amenities' has already existed in search
'amenities' has already existed in search
************ ALL DONE ************


In [15]:
len(fail_urls)#confirm all data parse is successful

0

In [16]:
# convert into DataFrame
urls = pd.DataFrame(succ_urls)

In [18]:
urls.head()

adults amenities     checkin    checkout children  \
0      3       NaN  2015-09-19  2015-09-20      NaN   
1      3       NaN  2015-09-14  2015-09-15      NaN   
2      2       NaN  2015-09-26  2015-09-27      NaN   
3      1       NaN  2015-09-02  2015-09-03      NaN   
4      3       NaN  2015-09-20  2015-09-29      NaN   

                          city couponCode customMaximumPriceFilter  \
0  New+York,+NY,+United+States        NaN                      NaN   
1       London,+United+Kingdom        NaN                      NaN   
2  New+York,+NY,+United+States        NaN                      175   
3         Hong+Kong,+Hong+Kong        NaN                      NaN   
4       London,+United+Kingdom        NaN                      275   

  customMinimumPriceFilter freeCancellation max_score min_score search_page  \
0                      NaN              NaN       NaN         4           1   
1                      NaN              NaN       NaN         4           1   
2                      NaN              NaN       NaN         5           1   
3                      NaN              NaN       NaN         4           1   
4                      NaN              NaN       NaN         5           1   

  stars_1 stars_2 stars_3 stars_4 stars_5  
0     NaN     NaN     NaN     yes     NaN  
1     NaN     NaN     yes     NaN     NaN  
2     NaN     NaN     NaN     yes     NaN  
3     NaN     NaN     NaN     yes     yes  
4     NaN     NaN     NaN     NaN     NaN

In [19]:
urls.describe()

adults amenities     checkin    checkout children  \
count   77677       704       77677       77677      260   
unique      4         8          29          48        2   
top         2  internet  2015-09-19  2015-09-20        1   
freq    45922       272        5279        6003      180   

                               city couponCode customMaximumPriceFilter  \
count                         77677       2221                    49288   
unique                            4          1                        6   
top     New+York,+NY,+United+States        yes                      175   
freq                          29384       2221                    14569   

       customMinimumPriceFilter freeCancellation max_score min_score  \
count                      1270            17949      5517     60826   
unique                        6                1         2         5   
top                         150              yes         5         4   
freq                        389            17949      5454     33639   

       search_page stars_1 stars_2 stars_3 stars_4 stars_5  
count        77677    9323    9931   14862   12704    4303  
unique          10       1       1       1       1       1  
top              1     yes     yes     yes     yes     yes  
freq         50000    9323    9931   14862   12704    4303

In [20]:
urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77677 entries, 0 to 77676
Data columns (total 18 columns):
adults                      77677 non-null object
amenities                   704 non-null object
checkin                     77677 non-null object
checkout                    77677 non-null object
children                    260 non-null object
city                        77677 non-null object
couponCode                  2221 non-null object
customMaximumPriceFilter    49288 non-null object
customMinimumPriceFilter    1270 non-null object
freeCancellation            17949 non-null object
max_score                   5517 non-null object
min_score                   60826 non-null object
search_page                 77677 non-null object
stars_1                     9323 non-null object
stars_2                     9931 non-null object
stars_3                     14862 non-null object
stars_4                     12704 non-null object
stars_5                     4303 non-null object
d

# Question 2
For each search query, how many amenities were selected? 

In [24]:
pd.notnull(urls.amenities).value_counts()

False    76973
True       704
Name: amenities, dtype: int64

In [23]:
urls.groupby('amenities').size()

amenities
breakfast                39
breakfast, yes_pet        1
internet                272
lounge                   22
shuttle                 111
yes_pet                  85
yes_smoking             170
yes_smoking, yes_pet      4
dtype: int64

In [28]:
def valueParse(val):
    if pd.isnull(val):
        return 0
    else:
        return len(val.split(", "))

In [36]:
urls['amenities'].apply(valueParse).value_counts().reset_index().rename(columns={"index":"amenities cnt","amenities":"record cnt"})

amenities cnt  record cnt
0              0       76973
1              1         699
2              2           5

# Define metrics

In [39]:
urls['search_page'].value_counts()

1     50000
2     11637
3      5864
4      3635
5      2422
6      1636
7      1114
8       740
9       436
10      193
Name: search_page, dtype: int64

In [78]:
url_pivot=urls.pivot_table(index=["city"],columns="search_page",values="checkin",aggfunc="count") 
url_pivot

search_page                                     1     10       2       3  \
city                                                                       
Hong+Kong,+Hong+Kong                      10735.0    NaN   766.0   168.0   
London,+United+Kingdom                    14775.0   89.0  5258.0  2876.0   
New+York,+NY,+United+States               16385.0  104.0  5329.0  2779.0   
San+Francisco,+California,+United+States   8105.0    NaN   284.0    41.0   

search_page                                    4       5      6      7      8  \
city                                                                            
Hong+Kong,+Hong+Kong                        62.0    27.0   13.0   10.0    4.0   
London,+United+Kingdom                    1833.0  1245.0  846.0  570.0  364.0   
New+York,+NY,+United+States               1728.0  1143.0  777.0  534.0  372.0   
San+Francisco,+California,+United+States    12.0     7.0    NaN    NaN    NaN   

search_page                                   9  
city                                             
Hong+Kong,+Hong+Kong                        1.0  
London,+United+Kingdom                    202.0  
New+York,+NY,+United+States               233.0  
San+Francisco,+California,+United+States    NaN

In [120]:
total_by_city=url_pivot.sum(axis=1)
total_by_city=pd.DataFrame(total_by_city)
total_by_city.reset_index(inplace=True)
total_by_city.rename(columns={0:'total_cnt'},inplace=True)

In [130]:
url_pivot1=url_pivot.reset_index()
url_pivot1=url_pivot1.merge(total_by_city,how="left")
url_pivot1['page1_rate']=url_pivot1['1']/url_pivot1['total_cnt']

In [131]:
url_pivot1

city        1     10       2       3  \
0                      Hong+Kong,+Hong+Kong  10735.0    NaN   766.0   168.0   
1                    London,+United+Kingdom  14775.0   89.0  5258.0  2876.0   
2               New+York,+NY,+United+States  16385.0  104.0  5329.0  2779.0   
3  San+Francisco,+California,+United+States   8105.0    NaN   284.0    41.0   

        4       5      6      7      8      9  total_cnt  page1_rate  
0    62.0    27.0   13.0   10.0    4.0    1.0    11786.0    0.910826  
1  1833.0  1245.0  846.0  570.0  364.0  202.0    28058.0    0.526588  
2  1728.0  1143.0  777.0  534.0  372.0  233.0    29384.0    0.557616  
3    12.0     7.0    NaN    NaN    NaN    NaN     8449.0    0.959285